In [1]:
library(caret, quiet = TRUE)
library(base64enc)
library(httr, quiet = TRUE)

library(mlbench)


Attaching package: ‘httr’

The following object is masked from ‘package:caret’:

    progress



# Build a Model

In [2]:
## https://www.machinelearningplus.com/machine-learning/caret-package/
## https://en.wikipedia.org/wiki/Multivariate_adaptive_regression_splines

data(BostonHousing)
BostonHousing$chas = as.numeric(BostonHousing$chas)

set.seed(1960)

dataset = BostonHousing[, -14] 

create_model  =  function() {

    model <- train(medv ~ ., data = BostonHousing, method = "earth")    

    return(model)
}


In [3]:
model = create_model()
# cat(model$feature_names)
# print(model)

Loading required package: earth
Loading required package: plotmo
Loading required package: plotrix
Loading required package: TeachingDemos


In [4]:
pred_labels <- predict(model, BostonHousing[, -14] , type="raw")
df = data.frame(BostonHousing[,14])
names(df) = c("medv")
df$Estimator = pred_labels
df$Error = df$Estimator - df$medv
MAPE = mean(abs(df$Error / df$medv))
summary(df)
MAPE

      medv          Estimator.y           Error.y       
 Min.   : 5.00   Min.   : 1.31908   Min.   :-14.688830  
 1st Qu.:17.02   1st Qu.:17.10733   1st Qu.: -1.687866  
 Median :21.20   Median :21.23878   Median :  0.036268  
 Mean   :22.53   Mean   :22.53281   Mean   :  0.000000  
 3rd Qu.:25.00   3rd Qu.:25.73923   3rd Qu.:  1.813339  
 Max.   :50.00   Max.   :49.10875   Max.   : 17.378345  

[1] 0.123243

# SQL Code Generation

In [5]:

test_ws_sql_gen = function(mod) {
    WS_URL = "https://sklearn2sql.herokuapp.com/model"
    WS_URL = "http://localhost:1888/model"
    model_serialized <- serialize(mod, NULL)
    b64_data = base64encode(model_serialized)
    data = list(Name = "xgboost_test_model", SerializedModel = b64_data , SQLDialect = "postgresql" , Mode="caret")
    r = POST(WS_URL, body = data, encode = "json")
    # print(r)
    content = content(r)
    # print(content)
    lSQL = content$model$SQLGenrationResult[[1]]$SQL # content["model"]["SQLGenrationResult"][0]["SQL"]
    return(lSQL);
}

In [6]:
lModelSQL = test_ws_sql_gen(model)
cat(lModelSQL)


WITH earth_input AS 
(SELECT "ADS"."KEY" AS "KEY", CAST("ADS"."Feature_0" AS FLOAT) AS "Feature_0", CAST("ADS"."Feature_1" AS FLOAT) AS "Feature_1", CAST("ADS"."Feature_2" AS FLOAT) AS "Feature_2", CAST("ADS"."Feature_3" AS FLOAT) AS "Feature_3", CAST("ADS"."Feature_4" AS FLOAT) AS "Feature_4", CAST("ADS"."Feature_5" AS FLOAT) AS "Feature_5", CAST("ADS"."Feature_6" AS FLOAT) AS "Feature_6", CAST("ADS"."Feature_7" AS FLOAT) AS "Feature_7", CAST("ADS"."Feature_8" AS FLOAT) AS "Feature_8", CAST("ADS"."Feature_9" AS FLOAT) AS "Feature_9", CAST("ADS"."Feature_10" AS FLOAT) AS "Feature_10", CAST("ADS"."Feature_11" AS FLOAT) AS "Feature_11", CAST("ADS"."Feature_12" AS FLOAT) AS "Feature_12" 
FROM "INPUT_DATA" AS "ADS"), 
earth_model_cte AS 
(SELECT earth_input."KEY" AS "KEY", -0.4507791371537364 * greatest(earth_input."Feature_12" - 6.12, 0) + 2.311188342915035 * greatest(6.12 - earth_input."Feature_12", 0) + 8.34586374702531 * greatest(earth_input."Feature_5" - 6.431, 0) + -0.623227227203093

# Execute the SQL Code

In [7]:
library(RODBC)
conn = odbcConnect("pgsql", uid="db", pwd="db", case="nochange")
odbcSetAutoCommit(conn , autoCommit = TRUE)

[1] 0

In [8]:
df_sql = dataset
names(df_sql) = sprintf("Feature_%d",0:(ncol(df_sql)-1))
df_sql$KEY = seq.int(nrow(dataset))

sqlDrop(conn , "INPUT_DATA" , errors = FALSE)
sqlSave(conn, df_sql, tablename = "INPUT_DATA", verbose = FALSE)

# df_sql

In [9]:
colnames(df_sql)
# odbcGetInfo(conn)
# sqlTables(conn)

[1] "Feature_0"  "Feature_1"  "Feature_2"  "Feature_3"  "Feature_4" 
 [6] "Feature_5"  "Feature_6"  "Feature_7"  "Feature_8"  "Feature_9" 
[11] "Feature_10" "Feature_11" "Feature_12" "KEY"

In [10]:
df_sql_out = sqlQuery(conn, lModelSQL)
head(df_sql_out[order(df_sql_out$KEY),])

KEY,Estimator
1,27.27768
2,21.68789
3,34.68358
4,37.51640
5,32.95109
6,27.60195


In [11]:
# df_sql_out

# R RPART Output

In [12]:
estimator  =  predict(model, dataset, type = "raw")
df_r_out = data.frame(estimator)
names(df_r_out) = c("Estimator")

df_r_out$KEY = seq.int(nrow(dataset))
head(df_r_out)


Estimator,KEY
27.27768,1
21.68789,2
34.68358,3
37.51640,4
32.95109,5
27.60195,6


# Compare R and SQL output

In [13]:
df_merge = merge(x = df_r_out, y = df_sql_out, by = "KEY", all = TRUE, , suffixes = c("_1","_2"))
head(df_merge)

KEY,Estimator_1,Estimator_2
1,27.27768,27.27768
2,21.68789,21.68789
3,34.68358,34.68358
4,37.51640,37.51640
5,32.95109,32.95109
6,27.60195,27.60195


In [14]:
df_merge$Error = df_merge$Estimator_1 - df_merge$Estimator_2
df_merge$AbsError = abs(df_merge$Error)
head(df_merge)


KEY,Estimator_1,Estimator_2,Error,AbsError
1,27.27768,27.27768,-1.065814e-14,1.065814e-14
2,21.68789,21.68789,-1.421085e-14,1.421085e-14
3,34.68358,34.68358,-2.842171e-14,2.842171e-14
4,37.51640,37.51640,-2.842171e-14,2.842171e-14
5,32.95109,32.95109,0.000000e+00,0.000000e+00
6,27.60195,27.60195,-2.486900e-14,2.486900e-14


In [15]:
df_merge_largest_errors = df_merge[df_merge$AbsError > 0.0001,]
df_merge_largest_errors

KEY,Estimator_1,Estimator_2,Error,AbsError


In [16]:
nrow(df_merge_largest_errors)
stopifnot(nrow(df_merge_largest_errors) == 0)


[1] 0

In [17]:
summary(df_sql_out)

      KEY          Estimator     
 Min.   :  1.0   Min.   : 1.319  
 1st Qu.:127.2   1st Qu.:17.107  
 Median :253.5   Median :21.239  
 Mean   :253.5   Mean   :22.533  
 3rd Qu.:379.8   3rd Qu.:25.739  
 Max.   :506.0   Max.   :49.109  

In [18]:
summary(df_r_out)

   Estimator           KEY       
 Min.   : 1.319   Min.   :  1.0  
 1st Qu.:17.107   1st Qu.:127.2  
 Median :21.239   Median :253.5  
 Mean   :22.533   Mean   :253.5  
 3rd Qu.:25.739   3rd Qu.:379.8  
 Max.   :49.109   Max.   :506.0  

In [19]:
summary(df_merge)

      KEY         Estimator_1      Estimator_2         Error           
 Min.   :  1.0   Min.   : 1.319   Min.   : 1.319   Min.   :-1.172e-13  
 1st Qu.:127.2   1st Qu.:17.107   1st Qu.:17.107   1st Qu.:-1.776e-14  
 Median :253.5   Median :21.239   Median :21.239   Median :-7.105e-15  
 Mean   :253.5   Mean   :22.533   Mean   :22.533   Mean   :-1.159e-14  
 3rd Qu.:379.8   3rd Qu.:25.739   3rd Qu.:25.739   3rd Qu.: 0.000e+00  
 Max.   :506.0   Max.   :49.109   Max.   :49.109   Max.   : 7.105e-14  
    AbsError        
 Min.   :0.000e+00  
 1st Qu.:3.553e-15  
 Median :8.882e-15  
 Mean   :1.578e-14  
 3rd Qu.:1.776e-14  
 Max.   :1.172e-13  

In [20]:
model$finalModel

Selected 20 of 24 terms, and 10 of 13 predictors
Termination condition: Reached nk 27
Importance: rm, lstat, ptratio, dis, nox, crim, tax, rad, b, indus, ...
Number of terms at each degree of interaction: 1 19 (additive model)
GCV 12.44743    RSS 5364.917    GRSq 0.853135    RSq 0.8744059

In [21]:
model$modelInfo

parameter,class,label
nprune,numeric,#Terms
degree,numeric,Product Degree


In [22]:
earth1 = model$finalModel

In [23]:
earth1$coefficients

,y
(Intercept),-25.419923898
h(lstat-6.12),-0.450779137
h(6.12-lstat),2.311188343
h(rm-6.431),8.345863747
h(ptratio-14.7),-0.623227227
h(14.7-ptratio),1.363852670
h(nox-0.488),-19.243816582
h(crim-4.42228),-0.831429478
h(dis-2.4358),-53.319514503
h(2.4358-dis),55.189153681


In [24]:
earth1$bx

(Intercept),h(lstat-6.12),h(6.12-lstat),h(rm-6.431),h(ptratio-14.7),h(14.7-ptratio),h(nox-0.488),h(crim-4.42228),h(dis-2.4358),h(2.4358-dis),h(tax-296),h(296-tax),h(dis-1.4165),h(crim-11.0874),h(rad-3),h(3-rad),h(4.93-indus),h(b-386.75),h(386.75-b),h(rm-7.929)
1,0.00,1.14,0.144,0.6,0,0.050,0,1.6542,0,0,0,2.6735,0,0,2,2.62,10.15,0.00,0
1,3.02,0.00,0.000,3.1,0,0.000,0,2.5313,0,0,54,3.5506,0,0,1,0.00,10.15,0.00,0
1,0.00,2.09,0.754,3.1,0,0.000,0,2.5313,0,0,54,3.5506,0,0,1,0.00,6.08,0.00,0
1,0.00,3.18,0.567,4.0,0,0.000,0,3.6264,0,0,74,4.6457,0,0,0,2.75,7.88,0.00,0
1,0.00,0.79,0.716,4.0,0,0.000,0,3.6264,0,0,74,4.6457,0,0,0,2.75,10.15,0.00,0
1,0.00,0.91,0.000,4.0,0,0.000,0,3.6264,0,0,74,4.6457,0,0,0,2.75,7.37,0.00,0
1,6.31,0.00,0.000,0.5,0,0.036,0,3.1247,0,15,0,4.1440,0,2,0,0.00,8.85,0.00,0
1,13.03,0.00,0.000,0.5,0,0.036,0,3.5147,0,15,0,4.5340,0,2,0,0.00,10.15,0.00,0
1,23.81,0.00,0.000,0.5,0,0.036,0,3.6463,0,15,0,4.6656,0,2,0,0.00,0.00,0.12,0
1,10.98,0.00,0.000,0.5,0,0.036,0,4.1563,0,15,0,5.1756,0,2,0,0.00,0.00,0.04,0


In [25]:
earth1$cuts

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat
(Intercept),0.00000,0,0.00,0,0.000,0.000,0,0.0000,0,0,0.0,0.00,0.00
h(lstat-6.12),0.00000,0,0.00,0,0.000,0.000,0,0.0000,0,0,0.0,0.00,6.12
h(6.12-lstat),0.00000,0,0.00,0,0.000,0.000,0,0.0000,0,0,0.0,0.00,6.12
h(rm-6.431),0.00000,0,0.00,0,0.000,6.431,0,0.0000,0,0,0.0,0.00,0.00
h(6.431-rm),0.00000,0,0.00,0,0.000,6.431,0,0.0000,0,0,0.0,0.00,0.00
h(ptratio-14.7),0.00000,0,0.00,0,0.000,0.000,0,0.0000,0,0,14.7,0.00,0.00
h(14.7-ptratio),0.00000,0,0.00,0,0.000,0.000,0,0.0000,0,0,14.7,0.00,0.00
h(nox-0.488),0.00000,0,0.00,0,0.488,0.000,0,0.0000,0,0,0.0,0.00,0.00
h(0.488-nox),0.00000,0,0.00,0,0.488,0.000,0,0.0000,0,0,0.0,0.00,0.00
h(crim-4.42228),4.42228,0,0.00,0,0.000,0.000,0,0.0000,0,0,0.0,0.00,0.00


In [26]:
earth1$dirs

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat
(Intercept),0,0,0,0,0,0,0,0,0,0,0,0,0
h(lstat-6.12),0,0,0,0,0,0,0,0,0,0,0,0,1
h(6.12-lstat),0,0,0,0,0,0,0,0,0,0,0,0,-1
h(rm-6.431),0,0,0,0,0,1,0,0,0,0,0,0,0
h(6.431-rm),0,0,0,0,0,-1,0,0,0,0,0,0,0
h(ptratio-14.7),0,0,0,0,0,0,0,0,0,0,1,0,0
h(14.7-ptratio),0,0,0,0,0,0,0,0,0,0,-1,0,0
h(nox-0.488),0,0,0,0,1,0,0,0,0,0,0,0,0
h(0.488-nox),0,0,0,0,-1,0,0,0,0,0,0,0,0
h(crim-4.42228),1,0,0,0,0,0,0,0,0,0,0,0,0
